In [37]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from sklearn.datasets import load_wine
import numpy as np

# Transformations
PyTorch has a lot of transforms that we can apply on our dataset:
- transforms for images  
- transforms for tensors  
- generic transforms (using lambda)
- custom transforms  
https://pytorch.org/vision/stable/transforms.html

In [38]:
# example of sequential with transformations
compose = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize([0.4914, 0.4822, 0.4465],
                                                   [0.2471, 0.2435, 0.2616])])

Here we are going to use dataset from last tutorial (№9). We **extend** it adding `transform` argument in `__init__`

In [39]:
class WineDataset(Dataset):
    def __init__(self, transform=None):
        # data loading
        self.dataset = load_wine()
        self.X = self.dataset.data.astype(np.float32)
        self.y = self.dataset.target.astype(np.float32)
        # we need column vector
        self.y = self.y.reshape(-1, 1)
        
        self.transform = transform
        
    def __getitem__(self, index):
        # dataset[index] -> (object, label/target)
        sample = (self.X[index], self.y[index])
        
        # apply transformations if it is available
        if self.transform:
            sample = self.transform(sample)
            
        return sample

    def __len__(self):
        # length of our dataset (number of samples)
        return self.X.shape[0]

Before using built-in classes for transformations, lets create  it by own

In [46]:
# we need to implement call method,
# because it is callable object
class ToTensor:
    def __call__(self, sample):
        inputs, targets = sample
        return (torch.from_numpy(inputs), torch.from_numpy(targets))

In [47]:
dataset_without_transformations = WineDataset()
inp, out = dataset_without_transformations[0]
type(inp), type(out)

(numpy.ndarray, numpy.ndarray)

In [48]:
dataset_with_transformations = WineDataset(transform=ToTensor())
inp, out = dataset_with_transformations[0]
type(inp), type(out)

(torch.Tensor, torch.Tensor)

In [55]:
# multiply tensor
class MulTransform:
    def __init__(self, factor):
        self.factor=factor
    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return (inputs, targets)
    
# composition of transforms
compose = transforms.Compose([ToTensor(),
                              MulTransform(factor=5)])

# apply it
dataset_with_compose_transformations = WineDataset(transform=compose)
inp_mul, out_mul = dataset_with_compose_transformations[0]

inp_mul == (inp * 5)

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True])